In [1]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('../data/raw/archive.zip')

In [21]:
df.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


In [22]:
df.shape

(308854, 19)

In [23]:
df_cleaned = df.copy()

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import SGDClassifier

In [25]:
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [26]:
cat_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False, drop='first'))
    ]
)

In [27]:
age_ord_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder())
    ]
)

In [28]:
health_ord_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder(categories=[['Poor','Fair','Good','Very Good','Excellent']]))
    ]
)

In [29]:
checkup_ord_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder(categories=[['Within the past year','Within the past 2 years','Within the past 5 years','5 or more years ago','Never']]))
    ]
)

In [30]:
num_cols = df_cleaned.select_dtypes(include=['float64']).columns

In [31]:
cat_cols = ['Arthritis', 'Depression', 'Diabetes', 'Exercise', 'Other_Cancer', 'Sex', 'Skin_Cancer', 'Smoking_History']

In [32]:
from sklearn.compose import ColumnTransformer

In [33]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat_cols', cat_transformer, cat_cols),
        ('age_col', age_ord_transformer, ['Age_Category']),
        ('health_col', health_ord_transformer, ['General_Health']),
        ('checkup_col', checkup_ord_transformer, ['Checkup'])
    ]
)

In [34]:
sgd_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('sgd', SGDClassifier(loss='log_loss', penalty='elasticnet', random_state=42))
    ]
)

In [35]:
target = df_cleaned['Heart_Disease'].map({'Yes': 1, 'No': 0})
sgd_pipe.fit(df_cleaned, target)

/Users/admin/Library/Caches/pypoetry/virtualenvs/adv-mla-lab-4-6rnbtT38-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption',
       'Fruit_Consumption', 'Green_Vegetables_Consumption',
       'FriedPotato_Consumption'],
      dtype='object')),
                                                 ('cat_cols',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='fir...
                                                                                               'Good',
                                                                                               'Very '
                                                                                               'Good',
                                                                                               'Excellent']]))]),
                                                  ['General_Health']),
                                                 ('checkup_col',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder(categories=[['Within '
                                                                                               'the '
                                                                                               'past '
                                                                                               'year',
                                                                                               'Within '
                                                                                               'the '
                                                                                               'past '
                                                                                               '2 '
                                                                                               'years',
                                                                                               'Within '
                                                                                               'the '
                                                                                               'past '
                                                                                               '5 '
                                                                                               'years',
                                                                                               '5 '
                                                                                               'or '
                                                                                               'more '
                                                                                               'years '
                                                                                               'ago',
                                                                                               'Never']]))]),
                                                  ['Checkup'])])),
                ('sgd',
                 SGDClassifier(loss='log_loss', penalty='elasticnet',
                               random_state=42))])

In [36]:
sgd_pipe.predict(df_cleaned)

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
obs = pd.DataFrame(df_cleaned.iloc[0]).transpose()
sgd_pipe.predict(obs)

array([0])